In [1]:
from IPython.display import HTML
HTML(open('../style.css', 'r').read())

In [2]:
%load_ext nb_mypy

Version 1.0.5


# The Usual Suspects

There has been a burglary at a jewelry store.  The [usual suspects](https://en.wikipedia.org/wiki/The_Usual_Suspects) have been arrested.  These are
<ul>
<li>Aaron,</li>
<li>Bernard, and</li>
<li>Caine.</li>
</ul>
Furthermore, the following facts have been established:
<ol>
<li>It is known that at least one of these suspects is indeed guilty.</li>
<li>If Aaron is guilty, he has exactly one accomplice.</li>
<li>If Bernard is innocent, then Caine is inncocent, too.</li>
<li>If exactly two of the suspects are guilty, then Caine is one of them.</li>
<li>If Caine is innocent, then Aaron is guilty.</li>
</ol>
It is our task to identify those suspects that are guilty.

Our first task is to define the set of propositional variables:
$$ \mathcal{P} := \{ \texttt{a}, \texttt{b}, \texttt{c} \} $$
The interpretation is that 
<ul>
<li>$\texttt{a}$ is true iff Aaron is guilty,</li> 
<li>$\texttt{b}$ is true iff Bernard is guilty, and</li>
<li>$\texttt{c}$ is true iff Caine is guilty.  </li>
</ul>

In [ ]:
P = { 'a', 'b', 'c' }

Our next task is to translate the facts given above into formulas from propositional logic. 

The statement "It is known that at least one of these suspects is indeed guilty." is translated as follows:
$$ \texttt{a} \vee \texttt{b} \vee \texttt{c}. $$ 

In [ ]:
f1 = 'a ∨ b ∨ c'

The statement "If Aaron is guilty, he has exactly one accomplice." is harder to translate into propositional logic. The idea is to split this statement into two statements:
* If Aaron is guilty, he has at least one accomplice.</li>
* If Aaron is guilty, he has at most  one accomplice.</li>

These statements can now be translated into the following formulas:

In [ ]:
f2 = 'a → b ∨ c'

In [ ]:
f3 = 'a → ¬(b ∧ c)'

The statement "If Bernard is innocent, then Caine is inncocent, too." is a simple implication:

In [ ]:
f4 = '¬b → ¬c'

The statement "If exactly two of the suspects are guilty, then Caine is one of them." is best translated into propositional logic by asking how this statement could be made false.
Obviously, this statement is false if two suspects are guilty, but Caine is innocent.
But this is only possible if Caine is innocent and Aaron and Bernard are true.  Hence we can translate this statement as follows:

In [ ]:
f5 = '¬(¬c ∧ a ∧ b)'

The statement "If Caine is innocent, then Aaron is guilty." is an implication:

In [ ]:
f6 = '¬c → a'

We define the set `Fs` of all formulas:

In [ ]:
Fs: set[str] = { f1, f2, f3, f4, f5, f6 }

We need to transform the strings <tt>f1</tt> to <tt>f6</tt> into nested tuples representing formulas.  To this end we import a parser for propositional formulas.

In [ ]:
%%capture
%run Propositional-Logic-Parser.ipynb

In [ ]:
from typing import TypeVar

In [ ]:
Formula = TypeVar('Formula')
Formula = str | tuple[Formula, ...]

In [ ]:
def parse(s: str) -> Formula:
    parser = LogicParser(s) # type: ignore
    return parser.parse()   # type ignore

Next, we transform all formulas into nested tuples:

In [ ]:
Gs = { parse(f) for f in Fs }
Gs

We are looking for a variable assignment $\mathcal{I}$ that satisfies all formulas in the set <tt>Fs</tt>.  As variable assignments are represented as subsets of the set $\mathcal{P}$ of propositional variables, we can just iterate over all subsets of $\mathcal{P}$.

The function `allSubsets(M)` takes a set `M` as its input and returns the list of all subsets of `M`.
The idea behind the definition of `allSubsets` is as follows:
1. Let `x` be any element from the set `M`. 
2. Then there are two kinds of subsets of `M`:
   * Those subsets $A \subseteq M$ that do not contain `x`.
   * Those subsets $B \subseteq M$ that do contain `x`.
3. The set $\mathcal{L}$ of those subsets `A` of `M` that do not contain `x` can be calculated recursively:
   $$ \mathcal{L} = \texttt{allSubsets}(M - \{x\}) $$
4. Adding `x` to the subsets in $\mathcal{L}$ yields all those subsets of $M$ that do contain `x`. 

In [ ]:
T = TypeVar('T')

In [ ]:
def allSubsets(M: set[T]) -> list[set[T]]:
    "Compute a list containing all subsets of the set M"
    if M == set():
        return [ set() ]
    x = M.pop() # remove x from M and return x
    L = allSubsets(M)
    return L + [ A | { x } for A in L ]

In [ ]:
allSubsets({1,2,3})

The function $\texttt{evaluate}(F, I)$ takes a propositional formula $F$ and a propositional variable assignment $I$ and evaluates $F$ using the assignment $I$.  We have discussed the details of this function previously.

In [ ]:
def evaluate(F: Formula, I: set[str]) -> bool:
    match F:
        case p if isinstance(p, str): 
            return p in I
        case ('⊤', ):     return True
        case ('⊥', ):     return False
        case ('¬', G):    return not evaluate(G, I)
        case ('∧', G, H): return     evaluate(G, I) and evaluate(H, I)
        case ('∨', G, H): return     evaluate(G, I) or  evaluate(H, I)
        case ('→', G, H): return     evaluate(G, I) <=  evaluate(H, I)
        case ('↔', G, H): return     evaluate(G, I) ==  evaluate(H, I)
    return None # type: ignore

The function `allTrue(Fs, I)` takes a set of propositional formula  `Fs`
and a propositional variable assignment `I`.  It returns `True` only if all formulas from `Fs` are 
`True` given the variable assignment `I`.

In [ ]:
def allTrue(Gs: set[Formula], I: set[str]) -> bool:
    return all([ evaluate(f, I) for f in Gs ])

Next, we compute the set of all variable assignments that render all formulas true:

In [ ]:
[ I for I in allSubsets(P) if allTrue(Gs, I) ]

It turns our that there is just one propositional variable assignment that satisfies all formulas from the set <tt>Fs</tt>.  Therefore, the problem has a unique solution: Bernard and Caine are guilty, while Aaron is innocent.